In [1]:
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import urllib2
from multiprocessing import Pool as ThreadPool
import math
import cPickle
import os 

In [2]:
priceArr = []
patCollect = []
endingInd = []
patLen = 10
futureE = 10

def percentChange(startPoint,currentPoint):
    try:
        x = ((float(currentPoint)-startPoint)/abs(startPoint))
        return x
    except:
        print(currentPoint, startPoint)
        return 0.00000000001

def loadPats():
    global priceArr
    
    priceArr = []
    with open(r"one.pickle", "rb") as input_file:
        e = cPickle.load(input_file)
        priceArr = e


def collectPats():
    global patCollect
    global endingInd
    patCollect = []
    endingInd = []
    for each in priceArr:
        tempCollect = []
        tempEnd = []
        sIndex = 0
        length = (len(each) - patLen - futureE)

        while sIndex < length:
            inc = 1
            tempPat = []
            while inc <= patLen:
                temp = percentChange(each[sIndex + inc -1], each[sIndex + inc])
                inc += 1
                tempPat.append(temp)
            tempCollect.append(tempPat)
            tempEnd.append(sIndex+patLen)
            sIndex += (patLen)
        patCollect.append(tempCollect)
        endingInd.append(tempEnd)

loadPats()
collectPats()

In [3]:
def yahooLoad(val):
    tempArr = []
    string = ""
    string = 'http://ichart.finance.yahoo.com/table.csv?s={0}'.format(val)

    csv = urllib2.urlopen(string).readlines()
    #for bar in xrange(1,min(len(csv),500)):
    for bar in xrange(1,min(50,len(csv))):
        close = csv[bar].split(',')[6]
        tempArr.append(float(close))
    tempArr = tempArr[::-1]
    #quoteCollect[val] = tempArr
    return {val: tempArr}


def currentPat(curArr):
    global patLen
    curPat = []
    sliceLen = patLen
    curr = curArr[-(sliceLen+1):]
    while curr[-2] == curr[-1]:
        sliceLen += 1
        curr = curArr[-(sliceLen+1):]
    i = 0
    while i < patLen:
        temp = percentChange(curr[i], curr[i + 1])
        curPat.append(temp)
        i += 1
    return curPat


def sortPats(array):
    finalArr = []
    def getKey(tup):
        return tup[1]
    finalArr = sorted(array, key=getKey)
    return finalArr

In [4]:
def matchPats(patCollect,endingInd,curPat):
    matchedPat = []
    matchedEndInd = []
    bestMatches = []
    for colInd,col in enumerate(patCollect):
        for rowInd,row in enumerate(col):
            mseCollect = []
            for ind,item in enumerate(row):
                mseTemp = abs(curPat[ind] - item)
                mseCollect.append(mseTemp)
            #mseAvg = np.average(weighting)
            mseAvg = np.average(mseCollect)
            bestMatches.append(([row,colInd,endingInd[colInd][rowInd]],mseAvg))
    test = sortPats(bestMatches)
    for item in test[:10]:
        matchedPat.append(item[0][0])
        matchedEndInd.append((item[0][1],item[0][2]))
    return matchedPat,matchedEndInd

In [5]:
def plotting(matchedPat,matchedEndInd):
    global futureE
    futureAverages = []
    stDev = []
    futureLine = []
    futurePercent = []
    if len(matchedPat) > 1:
        for i in matchedPat:
            tempPercent = []
            refInd = matchedPat.index(i)
            endInd = matchedEndInd[refInd]
            col = endInd[0]
            outcomesZ = priceArr[col][endInd[1]:(endInd[1]+patLen)]
            futureLine.append(outcomesZ)
            for x in outcomesZ:
                change = percentChange(priceArr[col][endInd[1]],x)
                tempPercent.append(change)
            futurePercent.append(tempPercent)

        futureAverages = []

        for arrItemNum in range(len(futurePercent[0])):
            tempOutcomes = []
            for arr in futurePercent:
                tempOutcomes.append(arr[arrItemNum])
            futureAverages.append(np.mean(tempOutcomes))
            stDev.append(np.std(tempOutcomes))

        return futureAverages,stDev

In [14]:
def sortBest(array):
    finalArr = []
    def getKey(val):
        return val['future'][-1]
    finalArr = sorted(array, key=getKey, reverse=True)
    return finalArr

bestCollect = []

def bestGo(val,curArr):
    global bestCollect

    #curArr = yahooLoad(val)
    curPat = currentPat(curArr)
    matchedPat,matchedEndInd = matchPats(patCollect,endingInd,curPat)
    futureAverages,stDev = plotting(matchedPat,matchedEndInd)
    totalDict = {'symbol': val,'matches': matchedPat,'current': curPat,'future': futureAverages, 'stDev': stDev, 'sharpe': futureAverages[-1]/stDev[-1]}
    bestCollect.append(totalDict)
    #except:
     #   bestCollect.append({'error':'error'})
    #  pass

In [15]:
arr = ['MMM','ABT','ABBV','ACN','ATVI','AYI','ADBE','AAP','AES','AET','AMG','AFL','A','GAS','APD','AKAM',
       'ALK','AA','ALXN','ALLE','AGN','ADS','ALL','GOOGL','GOOG','MO','AMZN','AEE','AAL','AEP','AXP','AIG','AMT']
#'AWK','AMP','ABC','AME','AMGN','APH','APC','ADI','ANTM','AON','APA','AIV','AAPL','AMAT','ADM','AJG','AIZ','T','ADSK','ADP',
#'AN','AZO','AVGO','AVB','AVY','BHI','BLL','BAC','BCR','BAX','BBT','BDX','BBBY','BRK-B','BBY','BIIB','BLK','HRB','BA','BWA','BXP',
#'BSX','BMY','BF-B']

In [16]:
pool = ThreadPool(4)
quoteCollect = pool.map(yahooLoad, arr)
pool.close()
pool.join()




In [ ]:
for i in quoteCollect: 
        for key,value in i.iteritems():
             bestGo(key,value)
   

 #finalBest = []
#for val in bestCollect:
#    if val['sharpe'] > 0.75:
#        finalBest.append(val)
#print(bestCollect)

submitArr = sortBest(bestCollect)

os.remove("best.pickle")
with open(r"best.pickle", "wb") as output_file:
    cPickle.dump(bestCollect, output_file)

In [ ]:
t = []
with open(r"best.pickle", "rb") as input_file:
        t = cPickle.load(input_file)
print(t)

In [ ]:
print(quoteCollect[0])